In [1]:
%load_ext autoreload

In [2]:
#from cap_package import SpotipyCollectPub as scp
from cap_package import ReadTransform as rt
from datetime import datetime
from dotenv import load_dotenv
import numpy as np
import os
from pathlib import Path

In [3]:
%autoreload 2

In [4]:
from IPython.utils.text import columnize
def disp_col(list_):
    
    # import -> from IPython.utils.text import columnize
    l = list(map(lambda x:repr(x)+',', list_))
    print(columnize(l, displaywidth=120))
def timer(start_time=None):
    if start_time is not None:
        return (datetime.now() - start_time).total_seconds()
    else:
        start_time = datetime.now()
        return start_time

In [5]:
load_dotenv()
path = Path(os.getenv('PATH_DATASET1.2'))

In [6]:
timbre_ = ['timbre_{}'.format(i+1) for i in range(12)]
pitch_ = ['pitch_{}'.format(i+1) for i in range(12)]

In [7]:
# Read dataset

path_ = path.joinpath('user_playlists')
pl_dataset = rt.read_dataset(path_)

print('Number of playlists:', len(pl_dataset))

Number of playlists: 13


In [8]:
# create data label list from playlist names
data_labels = []

for pl in pl_dataset:
   
    # extract playlist name
    name_ = pl[0] 
    # Number of tracks in this playlist
    num = len(pl[1]) 
    # create same labels for all tracks in this playlist
    # add to dataset labels
    data_labels += [name_] * len(pl[1])

print('\nFollowing playlists are in the dataset:\n')
disp_col(set(data_labels))


Following playlists are in the dataset:

'House-Trance',               'Classic progressive',   'Progressive 2',  'Our old school trance 2',
'Progressive 3',              'Progressive House',     'Progressive 5',
'Our old school trance 138',  'That familiar trance',  'Deep house',   
'Our old school trance',      'Progressive 1',         'Progressive 4',



In [9]:
# spilt columns for pitch and timbre vectors in segment dataframes
# Make separate list of track names and updated segments dataframes
tracks_name = []
tracks_seg = []
for pl in pl_dataset:
    
        for track in pl[1]:
            tracks_name.append(track[0])
            tracks_seg.append(rt.split_columns(track[1], pitch_cols=pitch_, timbre_cols=timbre_, ))

In [10]:
# remove any commas in track names; we will be using ',' as the delimiter for saving names in a csv
for i in range(len(tracks_name)):
    
    tracks_name[i] = tracks_name[i].replace(',', '')

In [11]:
if len(tracks_name) == len(tracks_seg) == len(data_labels):
    print("all good..............")

all good..............


In [12]:
# Check for null values in segments dataframe

nulls_in = []
for i in  range(len(tracks_seg)):
    n = tracks_seg[i].isnull().sum().sum()
    if n > 0:
        nulls_in.append(i)

In [13]:
# Make a mins list and a max list of timbre elements of each track

timbre_mins = []
timbre_maxs = []

for tr_seg in tracks_seg:
    
    min_ , max_ = rt.timbre_minmax_tr(tr_seg)
    timbre_mins.append(min_)
    timbre_maxs.append(max_)

In [14]:
# Evaluate population min and max of timbre elements

pop_timbre_min, pop_timbre_max = rt.pop_timbre_minmax(timbre_mins, timbre_maxs)

In [15]:
if len(timbre_mins) == len(timbre_maxs):
    print('Total number of tracks : ', len(timbre_mins))

if nulls_in:
    print('\nCheck "nulls_in" to see which tracks have missing values')
else:
    print('\nThere are no missing values in the dataset')
    
print('\nMinimum of timbre vec values for a sample track :\n ', timbre_mins[0])   
print('\nMaximum of timbre vec values for a sample track :\n ', timbre_maxs[0]) 

print('\nMinimums of timbre values over the whole dataset: \n{}'.format(pop_timbre_min))
print('\nMaximums of timbre values over the whole dataset: \n{}'.format(pop_timbre_max))

Total number of tracks :  372

There are no missing values in the dataset

Minimum of timbre vec values for a sample track :
  [31.792, -223.815, -80.134, -92.068, -25.921, -67.073, -51.277, -43.576, -51.169, -55.202, -47.284, -27.262]

Maximum of timbre vec values for a sample track :
  [53.999, 121.279, 103.483, 46.653, 119.55, 69.199, 52.258, 42.972, 34.68, 25.136, 29.058, 30.721]

Minimums of timbre values over the whole dataset: 
[0.0, -299.412, -252.321, -376.661, -102.82, -217.086, -125.704, -171.744, -94.96, -159.797, -224.671, -56.843]

Maximums of timbre values over the whole dataset: 
[60.121, 496.768, 318.383, 413.323, 191.311, 299.692, 121.598, 201.202, 76.953, 196.762, 192.364, 61.962]


In [16]:
data_arrays = rt.transform_dataset(tracks_seg, pop_timbre_min, pop_timbre_max, num_seg=100, bin_num=10)

In [17]:
print('Number of tracks :', len(data_arrays))
print('Length of an input array :', len(data_arrays[0]))
print('First 100 elements of a sample input array :\n')
disp_col(list(data_arrays[0][:100]))

Number of tracks : 372
Length of an input array : 2400
First 100 elements of a sample input array :

0.295,                 -0.2507954604945525,   -0.47711053918152113,   0.5148783287037806,     0.049,               
1.0,                   -0.15335189968560592,  -0.1457801996215009,    -0.06568112738325504,   0.215,               
0.585,                 -0.04088146078866273,  -0.038495442818901604,  -0.2604397375872606,    0.149,               
0.39,                  -0.24693405159715498,  -0.05069902881382293,   -0.0706191517802891,    0.028,               
0.078,                 0.291,                 -0.20257339468219404,   -0.24033848863261598,   0.5374661100114768,  
0.051,                 1.0,                   -0.21054299568935286,   -0.008804554373444629,  -0.09494084252304757,
0.087,                 0.628,                 -0.012078122939321534,  -0.003509878609958661,  -0.30422425635706074,
0.045,                 0.38,                  -0.22040318168427253,   -0.20836260477388

In [18]:
data_y, categories = rt.encode_label(data_labels)

In [19]:
categories

[array(['Classic progressive', 'Deep house', 'House-Trance',
        'Our old school trance', 'Our old school trance 138',
        'Our old school trance 2', 'Progressive 1', 'Progressive 2',
        'Progressive 3', 'Progressive 4', 'Progressive 5',
        'Progressive House', 'That familiar trance'], dtype='<U25')]

In [20]:
data = [i for i in zip(data_arrays, data_y)]

In [21]:
p = path.joinpath('usertracks_csv')
np.savetxt(p.joinpath('track_names.csv'), tracks_name, fmt='%s', delimiter = ',')
np.savetxt(p.joinpath('label_categories.csv'), categories, fmt='%s', delimiter = ',')

In [23]:
p1 = p.joinpath('track_arrays') 
for t in range(len(data)):
    np.savetxt(p1.joinpath('track_inp_{}.csv'.format(t)), data[t][0], fmt='%+10.3f', delimiter = ',')
    
p2 = p.joinpath('track_labels') 
for t in range(len(data)):
    np.savetxt(p2.joinpath('track_lab_{}.csv'.format(t)), data[t][1], fmt='%1.1f', delimiter = ',')
